In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadFromGCS").getOrCreate()

# Read json from GCS
California_review_df = spark.read.json("gs://msca-bdp-student-gcs/Group_5_final_project/review-California.json")
Illinois_review_df = spark.read.json("gs://msca-bdp-student-gcs/Group_5_final_project/review-Illinois.json")
New_York_review_df = spark.read.json("gs://msca-bdp-student-gcs/Group_5_final_project/review-New_York.json")

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1020bff7-68be-40d4-ac57-0f5acc872555;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.0 in spark-list
	found com.typesafe#config;1.4.2 in spark-list
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in spark-list
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in spark-list
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-j

In [3]:
# Read json from GCS
Meta_California = spark.read.json("gs://msca-bdp-student-gcs/Group_5_final_project/meta-California.json")
Meta_Illinois = spark.read.json("gs://msca-bdp-student-gcs/Group_5_final_project/meta-Illinois.json")
Meta_New_York = spark.read.json("gs://msca-bdp-student-gcs/Group_5_final_project/meta-New_York.json")

In [4]:
combined_review_df = California_review_df.unionByName(Illinois_review_df).unionByName(New_York_review_df)

review_column_to_exclude = ["pics"]
review_column_to_keep = [col for col in combined_review_df.columns if col not in review_column_to_exclude]

review_df = combined_review_df.select(review_column_to_keep)
review_df.show(5)

+--------------------+-----------------+------+----+--------------------+-------------+--------------------+
|             gmap_id|             name|rating|resp|                text|         time|             user_id|
+--------------------+-----------------+------+----+--------------------+-------------+--------------------+
|0x80c2c98c0e3c16f...|    Michael Rizal|     5|null|Great company. Am...|1599164133778|11316555113047622...|
|0x80c2c98c0e3c16f...|Faranak Rafizadeh|     5|null| Nice people
helpful|1618261672851|10122637137063761...|
|0x80c2c98c0e3c16f...|     Javier Perez|     5|null|                null|1524515066787|11116770366698148...|
|0x80c2c98c0e3c16f...|          Luis P.|     5|null|                null|1499739139293|11623088205734402...|
|0x80c2c98c0e3c16f...|   His Mama Cakez|     3|null|                null|1621410940638|10418165983558523...|
+--------------------+-----------------+------+----+--------------------+-------------+--------------------+
only showing top 5 

In [5]:
from pyspark.sql.functions import col

def flatten_misc(df):
    misc_cols = [c for c in df.select("MISC.*").columns]
    for mc in misc_cols:
        safe_mc = mc.replace(" ", "_").replace("&", "and")
        df = df.withColumn(f"MISC_{safe_mc}", col(f"`MISC`.`{mc}`"))
    return df.drop("MISC")

Meta_California = flatten_misc(Meta_California)
Meta_Illinois   = flatten_misc(Meta_Illinois)
Meta_New_York   = flatten_misc(Meta_New_York)

combined_meta_df = (
    Meta_California
    .unionByName(Meta_Illinois, allowMissingColumns=True)
    .unionByName(Meta_New_York, allowMissingColumns=True)
)

In [6]:
meta_column_to_exclude = ["url"]
meta_column_to_keep = [col for col in combined_meta_df.columns if col not in meta_column_to_exclude]

meta_df = combined_meta_df.select(meta_column_to_keep)
meta_df.show(5)

25/11/18 00:56:02 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+-------------------+-----------+--------------------+--------------------+----------+------------+--------------------+--------------+-----+--------------------+--------------------+--------------------+---------------+---------------+---------------+----------+-------------------+----------------------+-----------------+----------------------+---------------+--------------------+--------------+--------------------+-------------+----------------+--------------+--------------------+
|             address|avg_rating|           category|description|             gmap_id|               hours|  latitude|   longitude|                name|num_of_reviews|price|    relative_results|               state|  MISC_Accessibility|MISC_Activities| MISC_Amenities|MISC_Atmosphere|MISC_Crowd|MISC_Dining_options|MISC_From_the_business|MISC_Getting_here|MISC_Health_and_safety|MISC_Highlights|MISC_Lodging_options|MISC_Offerings|       MISC_Payments|MISC_Planning|MISC_Popular_f

In [7]:
review_df = review_df.withColumnRenamed("name", "cust_name")

In [8]:
meta_df = meta_df.withColumnRenamed("name", "store_name")

In [9]:
print("Review DF count:", review_df.count())
review_df.printSchema()

print("Meta DF count:", meta_df.count())
meta_df.printSchema()

Review DF count: 127086576
root
 |-- gmap_id: string (nullable = true)
 |-- cust_name: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)



Meta DF count: 967355
root
 |-- address: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- gmap_id: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- store_name: string (nullable = true)
 |-- num_of_reviews: long (nullable = true)
 |-- price: string (nullable = true)
 |-- relative_results: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)
 |-- MISC_Accessibility: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- MISC_Activities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- MISC_Amenities: array (nullable = true)
 |    |-- eleme

In [10]:
from pyspark.sql.functions import col

meta_df.groupBy("category") \
       .count() \
       .orderBy(col("count").desc()) \
       .show(20, truncate=False)

+------------------------------------------------------------------------------+-----+
|category                                                                      |count|
+------------------------------------------------------------------------------+-----+
|[Restaurant]                                                                  |14475|
|[Gas station]                                                                 |9725 |
|[Auto repair shop]                                                            |8870 |
|[Nail salon]                                                                  |8844 |
|[Park]                                                                        |8322 |
|[Mexican restaurant]                                                          |7991 |
|[Barber shop]                                                                 |7976 |
|[Park, Tourist attraction]                                                    |7575 |
|[Beauty salon]                            

In [11]:
from pyspark.sql.functions import expr

meta_df = meta_df.filter(
    expr("exists(category, x -> lower(x) LIKE '%restaurant%')")
)

In [12]:
# Allow very long strings to display
spark.conf.set("spark.sql.debug.maxToStringFields", 10000)
spark.conf.set("spark.sql.showStringLength", 1000000)

meta_df.select("address").show(3, truncate=False)

+-------------------------------------------------------+
|address                                                |
+-------------------------------------------------------+
|San Soo Dang, 761 S Vermont Ave, Los Angeles, CA 90005 |
|Vons Chicken, 12740 La Mirada Blvd, La Mirada, CA 90638|
|TACOS LA CABANA, 2015 22nd Ave, Oakland, CA 94606      |
+-------------------------------------------------------+
only showing top 3 rows



In [13]:
# Read ZIP code CSV from GCS
zip_df = spark.read.csv(
    "gs://msca-bdp-student-gcs/Group_5_final_project/zip_code_database.csv",
    header=True,
    inferSchema=False  # Optional: automatically detect types
)

zip_df = zip_df.withColumnRenamed("zip", "zip_lookup")\
                .withColumnRenamed("state", "zip_state")\
                .withColumnRenamed("county", "zip_county")\
                    .withColumnRenamed("primary_city", "zip_city")\
# Show sample rows
zip_df.show(10, truncate=False)

+----------+----------+---------+-------------------+------------------------+
|zip_lookup|zip_city  |zip_state|zip_county         |irs_estimated_population|
+----------+----------+---------+-------------------+------------------------+
|00501     |Holtsville|NY       |Suffolk County     |562                     |
|00544     |Holtsville|NY       |Suffolk County     |0                       |
|00601     |Adjuntas  |PR       |Adjuntas Municipio |0                       |
|00602     |Aguada    |PR       |Aguada Municipio   |0                       |
|00603     |Aguadilla |PR       |Aguadilla Municipio|0                       |
|00604     |Aguadilla |PR       |null               |0                       |
|00605     |Aguadilla |PR       |null               |0                       |
|00606     |Maricao   |PR       |Maricao Municipio  |0                       |
|00610     |Anasco    |PR       |Anasco Municipio   |0                       |
|00611     |Angeles   |PR       |Utuado Municipio   

In [14]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import re

# UDF to extract the last valid 5-digit zip in CA, NY, IL ranges
def extract_valid_zip(address):
    if not address:
        return None
    zips = re.findall(r'\b\d{5}\b', address)
    for z in reversed(zips):  # check from the end
        z_int = int(z)
        if 90000 <= z_int <= 96199:   # CA
            return z
        elif 10000 <= z_int <= 14999: # NY
            return z
        elif 60000 <= z_int <= 62999: # IL
            return z
    return None

extract_zip_udf = udf(extract_valid_zip, StringType())

meta_zip_df = meta_df.withColumn("zip", extract_zip_udf(col("address")))

meta_zip_df.select("address", "zip").show(10, truncate=False)

+-----------------------------------------------------------------------------------------------+-----+
|address                                                                                        |zip  |
+-----------------------------------------------------------------------------------------------+-----+
|San Soo Dang, 761 S Vermont Ave, Los Angeles, CA 90005                                         |90005|
|Vons Chicken, 12740 La Mirada Blvd, La Mirada, CA 90638                                        |90638|
|TACOS LA CABANA, 2015 22nd Ave, Oakland, CA 94606                                              |94606|
|Mariscos el poblano, 5401-5441 Coliseum Way, Oakland, CA 94601                                 |94601|
|Off The Hoof, 201 E 4th St, Santa Ana, CA 92701                                                |92701|
|La Potranca, 12821 Venice Blvd., Los Angeles, CA 90066                                         |90066|
|Atlantis Burgers, 5853 Imperial Hwy, South Gate, CA 90280      

In [15]:
from pyspark.sql.functions import regexp_extract, col

# 4. Left join meta_df_clean with zip_df
meta_zip_df = meta_zip_df.join(
    zip_df,
    meta_zip_df.zip == zip_df.zip_lookup,
    how="left"
)

# 5. Drop duplicate column and rename for clarity
meta_zip_df = meta_zip_df.drop("zip_lookup")


# 6. Show enriched data
meta_zip_df.select("address", "zip", "zip_city", "zip_state", "zip_county", "irs_estimated_population")\
    .show(10)

+--------------------+-----+-------------+---------+------------------+------------------------+
|             address|  zip|     zip_city|zip_state|        zip_county|irs_estimated_population|
+--------------------+-----+-------------+---------+------------------+------------------------+
|San Soo Dang, 761...|90005|  Los Angeles|       CA|Los Angeles County|                   27330|
|Vons Chicken, 127...|90638|    La Mirada|       CA|Los Angeles County|                   41100|
|TACOS LA CABANA, ...|94606|      Oakland|       CA|    Alameda County|                   27530|
|Mariscos el pobla...|94601|      Oakland|       CA|    Alameda County|                   40720|
|Off The Hoof, 201...|92701|    Santa Ana|       CA|     Orange County|                   40280|
|La Potranca, 1282...|90066|  Los Angeles|       CA|Los Angeles County|                   44640|
|Atlantis Burgers,...|90280|   South Gate|       CA|Los Angeles County|                   79010|
|Cowboy Burgers & ...|91706| B

In [16]:
from pyspark.sql.functions import col

# Top 20 states by count
meta_zip_df.groupBy("zip_state") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(20, truncate=False)

# Top 20 cities by count
meta_zip_df.groupBy("zip_city") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(20, truncate=False)

+---------+-----+
|zip_state|count|
+---------+-----+
|CA       |92493|
|NY       |50047|
|IL       |28771|
|null     |278  |
+---------+-----+



+-------------+-----+
|zip_city     |count|
+-------------+-----+
|New York     |10461|
|Chicago      |8035 |
|Los Angeles  |7101 |
|Brooklyn     |6198 |
|San Francisco|4248 |
|San Diego    |3960 |
|San Jose     |2122 |
|Bronx        |2053 |
|Sacramento   |1887 |
|Buffalo      |1373 |
|Fresno       |1261 |
|Long Beach   |1237 |
|Rochester    |1165 |
|Bakersfield  |1109 |
|Oakland      |1105 |
|Anaheim      |906  |
|Staten Island|874  |
|Flushing     |786  |
|Riverside    |773  |
|Santa Ana    |711  |
+-------------+-----+
only showing top 20 rows



In [17]:
from pyspark.sql.functions import col

# Show addresses where zip_state is null
meta_zip_df.filter(col("zip_state").isNull()) \
           .select("address", "store_name", "zip") \
           .show(20, truncate=False)

+--------------------------------------------------------+--------------------------------------------------------------+-----+
|address                                                 |store_name                                                    |zip  |
+--------------------------------------------------------+--------------------------------------------------------------+-----+
|null                                                    |Bull&Dragon                                                   |null |
|Yaki Tori, 6502 Westminster Blvd., Westminster, CA 92687|Yaki Tori                                                     |92687|
|Kirkwood Sunrise Grill, California                      |Kirkwood Sunrise Grill                                        |null |
|null                                                    |PowR Meals                                                    |null |
|null                                                    |Beans BBQ Catering - Santa Barbara            

In [18]:
from pyspark.sql.functions import col

meta_zip_df = meta_zip_df.filter(col("zip_state").isNotNull())

In [19]:
review_df = review_df.join(
    meta_zip_df.select("gmap_id").distinct(),
    on="gmap_id",
    how="left_semi"
)

In [20]:
print("Review DF count:", review_df.count())

Review DF count: 52215635


In [21]:
meta_df_duplicate_count = meta_zip_df.count() - meta_zip_df.dropDuplicates().count()

print("Meta_df Number of duplicated row: {}".format(meta_df_duplicate_count))

Meta_df Number of duplicated row: 250


In [22]:
review_df_duplicate_count = review_df.count() - review_df.dropDuplicates().count()

print("Review_df Number of duplicated row: {}".format(review_df_duplicate_count))

Review_df Number of duplicated row: 630060


In [23]:
meta_df_clean = meta_zip_df.dropDuplicates()
review_df_clean = review_df.dropDuplicates()

In [24]:
#Checking for missing values

from pyspark.sql import functions as F
from pyspark.sql.types import StructType, ArrayType

def missing_summary_safe(df):
    summary_cols = []
    for field in df.schema.fields:
        c = field.name
        dtype = field.dataType
        # For strings: check null only (or optionally check empty string separately)
        if isinstance(dtype, StructType) or isinstance(dtype, ArrayType):
            # Only check for null for complex types
            summary_cols.append(F.count(F.when(F.col(c).isNull(), c)).alias(c))
        elif "string" in str(dtype):
            summary_cols.append(F.count(F.when(F.col(c).isNull() | (F.col(c) == ""), c)).alias(c))
        else:
            # For numbers: just check nulls
            summary_cols.append(F.count(F.when(F.col(c).isNull(), c)).alias(c))
    return df.select(summary_cols)


In [25]:
missing_summary_safe(review_df_clean).show()
missing_summary_safe(meta_df_clean).show()

+-------+---------+------+--------+--------+----+-------+
|gmap_id|cust_name|rating|    resp|    text|time|user_id|
+-------+---------+------+--------+--------+----+-------+
|      0|        0| 20232|47425668|23302069|   0|  20232|
+-------+---------+------+--------+--------+----+-------+



+-------+----------+--------+-----------+-------+-----+--------+---------+----------+--------------+-----+----------------+-----+------------------+---------------+--------------+---------------+----------+-------------------+----------------------+-----------------+----------------------+---------------+--------------------+--------------+-------------+-------------+----------------+--------------+--------------------+---+--------+---------+----------+------------------------+
|address|avg_rating|category|description|gmap_id|hours|latitude|longitude|store_name|num_of_reviews|price|relative_results|state|MISC_Accessibility|MISC_Activities|MISC_Amenities|MISC_Atmosphere|MISC_Crowd|MISC_Dining_options|MISC_From_the_business|MISC_Getting_here|MISC_Health_and_safety|MISC_Highlights|MISC_Lodging_options|MISC_Offerings|MISC_Payments|MISC_Planning|MISC_Popular_for|MISC_Recycling|MISC_Service_options|zip|zip_city|zip_state|zip_county|irs_estimated_population|
+-------+----------+--------+-----

In [26]:
from pyspark.sql.functions import col, count

meta_df_clean.groupBy("price") \
             .agg(count("*").alias("num_businesses")) \
             .orderBy("num_businesses", ascending=False) \
             .show(truncate=False)

meta_df_clean.select("avg_rating").summary().show()

+-----+--------------+
|price|num_businesses|
+-----+--------------+
|$    |63251         |
|$$   |54145         |
|null |48447         |
|$$$  |3332          |
|₩₩   |666           |
|₩    |609           |
|$$$$ |571           |
|₩₩₩  |34            |
|₩₩₩₩ |6             |
+-----+--------------+



+-------+-------------------+
|summary|         avg_rating|
+-------+-------------------+
|  count|             171061|
|   mean|  4.198608683452113|
| stddev|0.42228319452638474|
|    min|                1.0|
|    25%|                4.0|
|    50%|                4.3|
|    75%|                4.5|
|    max|                5.0|
+-------+-------------------+



In [27]:
#final combined dataset
combined_df = review_df_clean.join(meta_df_clean, on="gmap_id", how="inner")

In [28]:
review_df.count()
meta_df.count()
combined_df.count()

51585575

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
import numpy as np

In [30]:
rating_dist = combined_df.filter(col("rating").isNotNull()) \
        .groupBy("rating") \
        .count() \
        .orderBy("rating")
    
total_ratings = combined_df.filter(col("rating").isNotNull()).count()
rating_dist = rating_dist.withColumn("percentage", round(col("count") / total_ratings * 100, 2))
rating_dist.show()

+------+--------+----------+
|rating|   count|percentage|
+------+--------+----------+
|     1| 3394122|      6.58|
|     2| 1934623|      3.75|
|     3| 4889422|      9.48|
|     4|11331461|     21.97|
|     5|30015715|     58.21|
+------+--------+----------+



In [31]:
combined_df.filter(col("rating").isNotNull()).select(
        count("rating").alias("total_reviews"),
        mean("rating").alias("mean"),
        stddev("rating").alias("std"),
        min("rating").alias("min"),
        max("rating").alias("max"),
        expr("percentile_approx(rating, 0.25)").alias("Q1"),
        expr("percentile_approx(rating, 0.5)").alias("median"),
        expr("percentile_approx(rating, 0.75)").alias("Q3"),
        skewness("rating").alias("skewness"),
        kurtosis("rating").alias("kurtosis")
    ).show()

+-------------+----------------+------------------+---+---+---+------+---+-------------------+------------------+
|total_reviews|            mean|               std|min|max| Q1|median| Q3|           skewness|          kurtosis|
+-------------+----------------+------------------+---+---+---+------+---+-------------------+------------------+
|     51565343|4.21476985036248|1.1718584391583498|  1|  5|  4|     5|  5|-1.5440452920252092|1.4337107888823093|
+-------------+----------------+------------------+---+---+---+------+---+-------------------+------------------+



In [32]:
# Binary classification
combined_df.filter(col("rating").isNotNull()) \
        .withColumn("binary_target", when(col("rating") >= 4, 1).otherwise(0)) \
        .groupBy("binary_target") \
        .count() \
        .withColumn("percentage", round(col("count") / total_ratings * 100, 2)) \
        .show()

+-------------+--------+----------+
|binary_target|   count|percentage|
+-------------+--------+----------+
|            1|41347176|     80.18|
|            0|10218167|     19.82|
+-------------+--------+----------+



In [33]:
combined_df.filter(col("rating").isNotNull()) \
        .withColumn("sentiment_3class",
                   when(col("rating") >= 4, "Positive")
                   .when(col("rating") == 3, "Neutral")
                   .otherwise("Negative")) \
        .groupBy("sentiment_3class") \
        .count() \
        .withColumn("percentage", round(col("count") / total_ratings * 100, 2)) \
        .orderBy("sentiment_3class") \
        .show()

+----------------+--------+----------+
|sentiment_3class|   count|percentage|
+----------------+--------+----------+
|        Negative| 5328745|     10.33|
|         Neutral| 4889422|      9.48|
|        Positive|41347176|     80.18|
+----------------+--------+----------+



In [34]:
combined_df.show(10)

+--------------------+---------------+------+----+--------------------+-------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+----------+------------+-------------------+--------------+-----+--------------------+-----+--------------------+---------------+--------------------+---------------+-----------------+--------------------+----------------------+-----------------+----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------+--------------------+-----+----------+---------+------------------+------------------------+
|             gmap_id|      cust_name|rating|resp|                text|         time|             user_id|             address|avg_rating|            category|         description|               hours|  latitude|   longitude|         store_name|num_of_reviews|price|    relative_results|state|  MISC_

In [35]:
# popularity rating - given based on the number of reviews

if "num_of_reviews" in combined_df.columns:
    combined_df.select("gmap_id", "num_of_reviews").distinct() \
        .filter(col("num_of_reviews").isNotNull()) \
        .select(
            count("num_of_reviews").alias("businesses"),
            mean("num_of_reviews").alias("mean"),
            stddev("num_of_reviews").alias("std"),
            min("num_of_reviews").alias("min"),
            max("num_of_reviews").alias("max"),
            expr("percentile_approx(num_of_reviews, 0.5)").alias("median"),
            expr("percentile_approx(num_of_reviews, 0.90)").alias("p90"),
            expr("percentile_approx(num_of_reviews, 0.95)").alias("p95")
        ).show()

    # Popularity categories
    print("\nPopularity Distribution:")
    combined_df.select("gmap_id", "num_of_reviews").distinct() \
        .withColumn("popularity_tier",
                   when(col("num_of_reviews") < 10, "Very Low (<10)")
                   .when(col("num_of_reviews") < 50, "Low (10-50)")
                   .when(col("num_of_reviews") < 100, "Medium (50-100)")
                   .when(col("num_of_reviews") < 500, "High (100-500)")
                   .otherwise("Very High (500+)")) \
        .groupBy("popularity_tier") \
        .count() \
        .orderBy("popularity_tier") \
        .show()

+----------+------------------+------------------+---+----+------+---+----+
|businesses|              mean|               std|min| max|median|p90| p95|
+----------+------------------+------------------+---+----+------+---+----+
|    171061|305.21879329595873|471.48817360567375|  1|9998|   158|734|1058|
+----------+------------------+------------------+---+----+------+---+----+


Popularity Distribution:


+----------------+-----+
| popularity_tier|count|
+----------------+-----+
|  High (100-500)|74786|
|     Low (10-50)|28810|
| Medium (50-100)|25214|
|Very High (500+)|30489|
|  Very Low (<10)|11762|
+----------------+-----+



In [64]:
distinct_business_count = combined_df.filter(col("price") == "₩") \
                                    .select('gmap_id', 'description', 'store_name', 'num_of_reviews','zip_city')\
                                    .distinct()

In [65]:
distinct_business_count.show(5)

+--------------------+--------------------+--------------------+--------------+----------+
|             gmap_id|         description|          store_name|num_of_reviews|  zip_city|
+--------------------+--------------------+--------------------+--------------+----------+
|0x80dcadefb3d9c28...|Charbroiled burge...|The Habit Burger ...|           320|    Colton|
|0x80c2955b2f7faf9...|Fast-food chain o...|       El Pollo Loco|           477|   Burbank|
|0x880ef84b0c30cb2...|Counter-serve spo...| Portillo's Hot Dogs|          1885|Naperville|
|0x80c2d2dbbb455fd...|Charbroiled burge...|The Habit Burger ...|           776|   Norwalk|
|0x880e2cae1b659e5...|Casual pizzeria f...|Dough Bros Pizzer...|           538|   Chicago|
+--------------------+--------------------+--------------------+--------------+----------+
only showing top 5 rows



In [47]:
#analysing text length of reviews
text_stats = text_df.withColumn("text_length", length(col("text"))) \
            .withColumn("word_count", size(split(col("text"), " ")))
        
print("\nText Statistics:")
text_stats.select(
    mean("text_length").alias("avg_chars"),
    stddev("text_length").alias("std_chars"),
    expr("percentile_approx(text_length, 0.5)").alias("median_chars"),
    mean("word_count").alias("avg_words"),
    expr("percentile_approx(word_count, 0.5)").alias("median_words"),
    min("text_length").alias("min_chars"),
    max("text_length").alias("max_chars")
).show()


Text Statistics:


+------------------+------------------+------------+-----------------+------------+---------+---------+
|         avg_chars|         std_chars|median_chars|        avg_words|median_words|min_chars|max_chars|
+------------------+------------------+------------+-----------------+------------+---------+---------+
|127.93216905287484|186.83825373196024|          68|23.23638388394989|          12|        1|     9637|
+------------------+------------------+------------+-----------------+------------+---------+---------+



In [48]:
#check if negative reviews have longer review length
negative_avg = text_stats.filter(col("rating") <= 2).select(mean("text_length")).collect()[0][0]
positive_avg = text_stats.filter(col("rating") >= 4).select(mean("text_length")).collect()[0][0]

In [49]:
if "zip_state" in combined_df.columns:
        print("\nState Distribution:")
        state_stats = combined_df.groupBy("zip_state") \
            .agg(
                count("*").alias("review_count"),
                countDistinct("gmap_id").alias("business_count"),
                mean("rating").alias("avg_rating")
            ) \
            .orderBy(desc("review_count"))
        
        print(f"\nTotal states: {state_stats.count()}")
        print("\nTop 15 States by Review Volume:")
        state_stats.limit(15).show()


State Distribution:



Total states: 3

Top 15 States by Review Volume:


+---------+------------+--------------+-----------------+
|zip_state|review_count|business_count|       avg_rating|
+---------+------------+--------------+-----------------+
|       CA|    29208458|         92366|4.230422235698517|
|       NY|    12970086|         49956|4.202085184379853|
|       IL|     9407031|         28739|4.183652011176494|
+---------+------------+--------------+-----------------+



In [51]:
print("\nGeographic Rating Variance (for spatial features):")
state_stats.select(
    variance("avg_rating").alias("variance"),
    stddev("avg_rating").alias("std_dev"),
    min("avg_rating").alias("min_state_rating"),
    max("avg_rating").alias("max_state_rating")
).show()


Geographic Rating Variance (for spatial features):


+--------------------+--------------------+-----------------+-----------------+
|            variance|             std_dev| min_state_rating| max_state_rating|
+--------------------+--------------------+-----------------+-----------------+
|5.550373756036713E-4|0.023559231218434767|4.183652011176494|4.230422235698517|
+--------------------+--------------------+-----------------+-----------------+



In [52]:
if "irs_estimated_population" in combined_df.columns:
        print("\nPopulation Context:")
        combined_df.filter(col("irs_estimated_population").isNotNull()) \
            .select("gmap_id", "irs_estimated_population", "avg_rating") \
            .distinct() \
            .select(
                mean("irs_estimated_population").alias("avg_population"),
                min("irs_estimated_population").alias("min_pop"),
                max("irs_estimated_population").alias("max_pop"),
                expr("percentile_approx(irs_estimated_population, 0.5)").alias("median_pop")
            ).show()


Population Context:


+------------------+-------+-------+----------+
|    avg_population|min_pop|max_pop|median_pop|
+------------------+-------+-------+----------+
|32650.882264221535|      0|   9980|   30310.0|
+------------------+-------+-------+----------+



In [55]:
if "price" in combined_df.columns:
        print("\nPrice Distribution:")
        combined_df.groupBy("price") \
            .agg(
                count("*").alias("review_count"),
                mean("rating").alias("avg_rating"),
                stddev("rating").alias("std_rating")
            ) \
            .orderBy("price") \
            .show()
        
        # Price completeness
price_complete = combined_df.filter(col("price").isNotNull()).count()
print(f"Price data completeness: {price_complete/combined_df.count()*100:.2f}%")


Price Distribution:


+-----+------------+------------------+------------------+
|price|review_count|        avg_rating|        std_rating|
+-----+------------+------------------+------------------+
| null|     5099140| 4.306061702087025| 1.138854721830487|
|    $|    21303158|4.0992910065258865| 1.252795289051027|
|   $$|    22302748|4.2855594229990865|1.1022352047914723|
|  $$$|     1512729| 4.411286489516629|0.9953814144864344|
| $$$$|      305768| 4.497903639360561|0.9669886665576052|
|    ₩|      445551|4.1536389773561275| 1.221377310638883|
|   ₩₩|      564923| 4.302586370177883| 1.084138391083157|
|  ₩₩₩|       40011| 4.419059758566394|0.9947955675150775|
| ₩₩₩₩|       11547| 4.492075863860743|0.9749159931094488|
+-----+------------+------------------+------------------+



Price data completeness: 90.12%


In [56]:
if "num_of_reviews" in combined_df.columns:
    print("\nReview Volume Distribution:")
    combined_df.select("gmap_id", "num_of_reviews", "avg_rating").distinct() \
        .withColumn("volume_bin",
                   when(col("num_of_reviews") < 10, "0-10")
                   .when(col("num_of_reviews") < 50, "10-50")
                   .when(col("num_of_reviews") < 100, "50-100")
                   .when(col("num_of_reviews") < 500, "100-500")
                   .otherwise("500+")) \
        .groupBy("volume_bin") \
        .agg(
            count("*").alias("business_count"),
            mean("avg_rating").alias("avg_rating")
        ) \
        .orderBy("volume_bin") \
        .show()




Review Volume Distribution:


+----------+--------------+------------------+
|volume_bin|business_count|        avg_rating|
+----------+--------------+------------------+
|      0-10|         11762|4.1103468797823535|
|     10-50|         28810| 4.160173550850395|
|   100-500|         74786| 4.227257775519481|
|    50-100|         25214|4.2028515903862935|
|      500+|         30489| 4.195194988356449|
+----------+--------------+------------------+



In [57]:
# Correlation: num_reviews vs avg_rating
corr = combined_df.filter(
    col("num_of_reviews").isNotNull() & col("avg_rating").isNotNull()
).select("gmap_id", "num_of_reviews", "avg_rating").distinct() \
    .stat.corr("num_of_reviews", "avg_rating")
print(f"Correlation (num_reviews vs avg_rating): {corr:.4f}")

Correlation (num_reviews vs avg_rating): 0.0321


In [59]:
if "irs_estimated_population" in combined_df.columns:
    print("\n--- Population Density Effects ---")

    pop_bins = combined_df.select("gmap_id", "irs_estimated_population", "avg_rating").distinct() \
        .filter(col("irs_estimated_population").isNotNull()) \
        .withColumn("pop_category",
                   when(col("irs_estimated_population") < 10000, "Rural (<10K)")
                   .when(col("irs_estimated_population") < 50000, "Small Town (10K-50K)")
                   .when(col("irs_estimated_population") < 100000, "Mid-Size (50K-100K)")
                   .otherwise("Urban (>100K)"))

    pop_bins.groupBy("pop_category") \
        .agg(
            count("*").alias("business_count"),
            mean("avg_rating").alias("avg_rating"),
            stddev("avg_rating").alias("std_rating")
        ) \
        .orderBy("pop_category") \
        .show()


--- Population Density Effects ---


+--------------------+--------------+-----------------+-------------------+
|        pop_category|business_count|       avg_rating|         std_rating|
+--------------------+--------------+-----------------+-------------------+
| Mid-Size (50K-100K)|         31489|4.168814506653115|0.41713626404012777|
|        Rural (<10K)|         21301|4.226712360922022|0.46233844617246816|
|Small Town (10K-50K)|        118172|4.201578208035755|0.41562100752416264|
|       Urban (>100K)|            99|4.083838383838384| 0.3477939428381587|
+--------------------+--------------+-----------------+-------------------+



In [1]:
#grouped by time/open
combined_df.groupBy("state") \
        .count() \
        .orderBy(desc("count")) \
        .withColumn("percentage", round(col("count") / total_businesses * 100, 2)) \
        .show()

NameError: name 'combined_df' is not defined

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadGCS").getOrCreate()

business_df = spark.read.parquet("gs://msca-bdp-student-gcs/Group_5_final_project/store_df/")
business_df.show(3)

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e02a5e7b-8c98-452b-93a0-3c6b905c36de;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.0 in spark-list
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in spark-list
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in spark-list
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in spark-list
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in spark-list
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in spark-list
	found com.google.cloud#google-cloud-storage;2.16.0 in spark-list
	found com.google.guava#gua

+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------+-----+--------------------+-------------------+--------------------+---------------+--------------------+---------------+-----------------+--------------------+----------------------+-----------------+----------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-----+--------------+---------+---------------+------------------------+----------------+-------------+-----------------------+------------------+------------+
|             gmap_id|             address|avg_rating|            category|         description|               hours|          latitude|         longitude|          store_name|num_of_reviews|price|    relative_results|              state|  MISC_Accessibility|MISC_Acti

In [3]:
biz_closure_prob_df = spark.read.parquet("gs://msca-bdp-student-gcs/Group_5_final_project/biz_closure_prob_df/")

biz_closure_prob_df.show(3)

+--------------------+--------------------+----------+--------------------+
|             gmap_id|          store_name|prediction|        closure_prob|
+--------------------+--------------------+----------+--------------------+
|0x89e85a90fa80ea7...|            Checkers|       0.0|0.007217116418191323|
|0x89c2c219189e518...|   Momo Asian Fusion|       0.0|0.017988535282472868|
|0x8085b6a8ebae49f...|Rosso Pizzeria & ...|       1.0|  0.9591585341045397|
+--------------------+--------------------+----------+--------------------+
only showing top 3 rows



In [5]:
from pyspark.sql.functions import col

# Select only the columns we want to join
biz_prob_subset = biz_closure_prob_df.select("gmap_id", "prediction", "closure_prob")

# Left join with original business DataFrame
business_with_prob_df = business_df.join(
    biz_prob_subset,
    on="gmap_id",
    how="left"
)

# Preview the result
business_with_prob_df.show(3, truncate=False)


+-------------------------------------+------------------------------------------------------------------------+----------+----------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+------------------+-------------------------------+--------------+-----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------+---------------+----------------------------+---------------+--------

In [6]:
from pyspark.sql import functions as F

# =========================
# Parameters for shrinkage
# =========================
k = 10  # pseudo-count (adjust this to control shrinkage strength)

# global mean of predicted sentiment across all stores
global_mean = business_with_prob_df.agg(F.mean("avg_predicted_sentiment")).collect()[0][0]

# =========================
# Compute shrunk sentiment
# =========================
business_with_prob_df = business_with_prob_df.withColumn(
    "shrunk_sentiment",
    (F.col("avg_predicted_sentiment") * F.col("review_count") + global_mean * k) /
    (F.col("review_count") + k)
)

# =========================
# Show results
# =========================
business_with_prob_df.select(
    "store_name",
    "avg_predicted_sentiment",
    "review_count",
    "shrunk_sentiment",
    "avg_rating"
).show(10, truncate=False)


+-------------------------------+-----------------------+------------+------------------+----------+
|store_name                     |avg_predicted_sentiment|review_count|shrunk_sentiment  |avg_rating|
+-------------------------------+-----------------------+------------+------------------+----------+
|Checkers                       |3.1702048057536985     |11          |3.629562377589972 |3.5       |
|Momo Asian Fusion              |4.58498825475099       |5           |4.284899889323579 |4.6       |
|Rosso Pizzeria & Mozzarella Bar|3.864661623144518      |5           |4.044791012121421 |4.2       |
|Junior's Restaurant & Bakery   |4.290628785509187      |15          |4.228319553949461 |4.4       |
|Woori Village                  |3.1132834520223933     |10          |3.624069579316133 |3.5       |
|The Flame Broiler              |4.007555898949703      |5           |4.092422437389816 |3.8       |
|200 Fifth                      |4.116263523339339      |14          |4.124010266368728 |4.

In [7]:
from pyspark.sql.functions import col, sum as spark_sum, lit
from pyspark.sql import Row

# Step 1: compute missing counts (one row)
missing_df = business_with_prob_df.select([
    spark_sum(col(c).isNull().cast("int")).alias(c)
    for c in business_with_prob_df.columns
])

# Step 2: collect as Python dict
missing_dict = missing_df.first().asDict()

# Step 3: convert to list of Rows
rows = [Row(column=col_name, missing_count=missing_dict[col_name])
        for col_name in missing_dict]

# Step 4: create a transposed Spark DataFrame
transposed_missing_df = spark.createDataFrame(rows)

# Step 5: show results
transposed_missing_df.show(50, truncate=False)

+------------------------+-------------+
|column                  |missing_count|
+------------------------+-------------+
|gmap_id                 |0            |
|address                 |0            |
|avg_rating              |0            |
|category                |0            |
|description             |72284        |
|hours                   |8333         |
|latitude                |0            |
|longitude               |0            |
|store_name              |0            |
|num_of_reviews          |0            |
|price                   |47163        |
|relative_results        |12725        |
|state                   |46186        |
|MISC_Accessibility      |33726        |
|MISC_Activities         |169478       |
|MISC_Amenities          |26983        |
|MISC_Atmosphere         |38927        |
|MISC_Crowd              |52454        |
|MISC_Dining_options     |50973        |
|MISC_From_the_business  |163447       |
|MISC_Getting_here       |169478       |
|MISC_Health_and

In [8]:
cols_to_drop = [
    "MISC_Activities", "MISC_From_the_business", "MISC_Getting_here",
    "MISC_Health_and_safety", "MISC_Lodging_options", "MISC_Recycling"
    "review_count", 
]

business_with_prob_df_clean = business_with_prob_df.drop(*cols_to_drop)

In [9]:
from pyspark.sql.types import DoubleType

business_with_prob_df_clean = business_with_prob_df_clean.withColumn("irs_estimated_population", col("irs_estimated_population").cast(DoubleType()))

In [10]:
from pyspark.sql.functions import explode, col, when, array_contains

misc_cols = [
    "MISC_Accessibility", "MISC_Amenities", "MISC_Atmosphere",
    "MISC_Crowd", "MISC_Dining_options", "MISC_Offerings",
    "MISC_Payments", "MISC_Planning", "MISC_Popular_for",
    "MISC_Service_options", "MISC_Highlights"
]

for c in misc_cols:
    # Explode the array to count frequency
    top_items = (business_with_prob_df_clean
                 .select(explode(col(c)).alias("item"))
                 .groupBy("item")
                 .count()
                 .orderBy(col("count").desc())
                 .limit(3)
                 .collect())
    
    top_items_list = [row["item"] for row in top_items if row["item"] is not None]

    # Create one-hot columns for top 3
    for item in top_items_list:
        col_name = f"{c}_{item.replace(' ', '_')}_flag"
        business_with_prob_df_clean = business_with_prob_df_clean.withColumn(
            col_name,
            when(array_contains(col(c), item), 1).otherwise(0)
        )

In [11]:
business_with_prob_df_clean.show(3)

+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------------+------------------+--------------------+--------------+-----+--------------------+-------------------+--------------------+--------------------+---------------+-----------------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-----+--------------+---------+---------------+------------------------+----------------+-------------+-----------------------+------------------+------------+----------+--------------------+-----------------+------------------------------------------------------+------------------------------------------------------+---------------------------------------------------------+---------------------------------+-------------------------------+----------------------------+---------------------------+-------------------------+---

In [12]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from graphframes import GraphFrame
import pandas as pd

In [13]:
business_with_prob_df_clean = business_with_prob_df_clean.withColumn(
    "success_score",
    (
        F.col("shrunk_sentiment") * 0.4 +  # Sentiment weight
        F.col("avg_rating") * 0.3 +         # Rating weight
        (1 - F.col("closure_prob")) * 0.3   # Survival probability weight
    )
)

In [16]:
business_with_prob_df_clean.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- address: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- store_name: string (nullable = true)
 |-- num_of_reviews: long (nullable = true)
 |-- price: string (nullable = true)
 |-- relative_results: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)
 |-- MISC_Accessibility: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- MISC_Amenities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- MISC_Atmosphere: array (nullable = true)
 |    |-- element: string (containsNu

In [17]:
business_with_prob_df_clean = business_with_prob_df_clean.withColumn(
    "neighborhood",
    F.when(F.size(F.col("relative_results")) > 0, 
           F.col("relative_results")[0])
    .otherwise(None)
)

print(f"Total records: {business_with_prob_df_clean.count()}")
print(f"Records with neighborhood: {business_with_prob_df_clean.filter(F.col('neighborhood').isNotNull()).count()}")


Total records: 169478


Records with neighborhood: 156753


In [18]:
business_with_prob_df_clean.select("store_name", "relative_results", "neighborhood").show(5, truncate=False)


+-------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|store_name                     |relative_results                                                                                                                                                                                   |neighborhood                         |
+-------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|Checkers                       |[0x89e85a90fe4fe409:0xf0efb24d795b110f, 0x89e85a8dc34940cb:0xcd817f3355bad91, 0x89e85a91961bb75f:0xf54aba241f7dc05b, 0x89e85a90ff1ef4cf:0x10d0beb59133f19, 0x89e85a

In [19]:
key_cols = ["shrunk_sentiment", "avg_rating", "closure_prob", "neighborhood"]
for col_name in key_cols:
    null_count = business_with_prob_df_clean.filter(F.col(col_name).isNull()).count()
    print(f"{col_name}: {null_count} nulls")


shrunk_sentiment: 0 nulls


avg_rating: 0 nulls


closure_prob: 0 nulls


neighborhood: 12725 nulls


In [20]:
business_with_prob_df_clean = business_with_prob_df_clean.withColumn(
    "success_score",
    F.when(
        F.col("shrunk_sentiment").isNotNull() & 
        F.col("avg_rating").isNotNull() & 
        F.col("closure_prob").isNotNull(),
        (
            F.col("shrunk_sentiment") * 0.4 +  # Sentiment weight
            F.col("avg_rating") * 0.3 +         # Rating weight
            (1 - F.col("closure_prob")) * 0.3   # Survival probability weight
        )
    ).otherwise(None)
)

print(f"\nRecords with success_score: {business_with_prob_df_clean.filter(F.col('success_score').isNotNull()).count()}")


Records with success_score: 169478


In [21]:
valid_businesses = business_with_prob_df_clean.filter(
    F.col("neighborhood").isNotNull() & 
    F.col("success_score").isNotNull()
)

print(f"Valid businesses for aggregation: {valid_businesses.count()}")


Valid businesses for aggregation: 156753


In [22]:
neighborhood_stats = valid_businesses.groupBy("neighborhood").agg(
    F.count("*").alias("num_businesses"),
    F.avg("success_score").alias("avg_success_score"),
    F.avg("shrunk_sentiment").alias("avg_sentiment"),
    F.avg("avg_rating").alias("avg_rating"),
    F.avg("closure_prob").alias("avg_closure_prob"),
    F.avg("irs_estimated_population").alias("avg_population"),
    F.sum(F.when(F.col("price_numeric").isNotNull(), F.col("price_numeric")).otherwise(0)).alias("total_price_level")
).filter(F.col("num_businesses") >= 3)  # Only neighborhoods with at least 3 businesses


In [23]:
print(f"\nTotal neighborhoods with valid data: {neighborhood_stats.count()}")
print("\nTop Neighborhoods by Success Score:")
neighborhood_stats.orderBy(F.col("avg_success_score").desc()).show(20, truncate=False)



Total neighborhoods with valid data: 18646

Top Neighborhoods by Success Score:


+-------------------------------------+--------------+------------------+------------------+------------------+--------------------+------------------+-----------------+
|neighborhood                         |num_businesses|avg_success_score |avg_sentiment     |avg_rating        |avg_closure_prob    |avg_population    |total_price_level|
+-------------------------------------+--------------+------------------+------------------+------------------+--------------------+------------------+-----------------+
|0x882d528b8dc8c5fd:0xee7a47353dedd292|3             |3.4794660939924498|4.3854929378722805|4.766666666666667 |0.01577027052154068 |2063.3333333333335|2                |
|0x80d9592e5a88e653:0x84581a39541b9cd2|3             |3.470650108358123 |4.345155700986275 |4.8               |0.024707240121291815|36810.0           |3                |
|0x80dcd882045441c5:0x4674b34f80642d94|3             |3.4574226898961626|4.340550201537546 |4.766666666666667 |0.029324635729519363|29906.666666666668

In [24]:
vertices = neighborhood_stats.withColumnRenamed("neighborhood", "id")
vertices = vertices.withColumn("node_type", F.lit("neighborhood"))

print(f"Total neighborhoods (nodes): {vertices.count()}")
vertices.show(5, truncate=False)

Total neighborhoods (nodes): 18646


+-------------------------------------+--------------+------------------+------------------+------------------+--------------------+-----------------+-----------------+------------+
|id                                   |num_businesses|avg_success_score |avg_sentiment     |avg_rating        |avg_closure_prob    |avg_population   |total_price_level|node_type   |
+-------------------------------------+--------------+------------------+------------------+------------------+--------------------+-----------------+-----------------+------------+
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|3             |2.866328797458913 |3.7374668827857724|3.6999999999999997|0.1288598521846532  |42000.0          |3                |neighborhood|
|0x808f8fbfe8f43d0b:0xeb6a722235dbdaaa|7             |3.058570685321204 |4.061943144268407 |3.9428571428571426|0.1635457174776749  |40888.57142857143|7                |neighborhood|
|0x80dd2c26d780ad17:0xbdb79faa79b26dd3|3             |3.3098743450589274|4.207878799425834

In [25]:
neighborhood_pairs = neighborhood_stats.alias("n1").crossJoin(
    neighborhood_stats.alias("n2")
).filter(
    F.col("n1.neighborhood") < F.col("n2.neighborhood")  # Avoid duplicates
)

In [26]:
edges = neighborhood_pairs.withColumn(
    "similarity_score",
    (
        1 - F.abs(F.col("n1.avg_success_score") - F.col("n2.avg_success_score")) / 5 * 0.3 +
        1 - F.abs(F.col("n1.avg_sentiment") - F.col("n2.avg_sentiment")) / 5 * 0.3 +
        1 - F.abs(F.col("n1.avg_rating") - F.col("n2.avg_rating")) / 5 * 0.4
    )
).filter(F.col("similarity_score") > 0.7)  # Only keep similar neighborhoods

edges = edges.select(
    F.col("n1.neighborhood").alias("src"),
    F.col("n2.neighborhood").alias("dst"),
    F.col("similarity_score").alias("relationship")
)

print(f"Total edges (connections): {edges.count()}")
edges.show(10, truncate=False)


Total edges (connections): 173827335


+-------------------------------------+-------------------------------------+------------------+
|src                                  |dst                                  |relationship      |
+-------------------------------------+-------------------------------------+------------------+
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|0x80dd2c26d780ad17:0xbdb79faa79b26dd3|2.886495885478929 |
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|0x8875ddde2e71b9c9:0x4010a5eea0e0f6b8|2.925011850247585 |
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|0x89d77ccec0e80fe1:0x7dd7e39681274ca8|2.9671751131789366|
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|0x880fa4e9ab912b73:0x61da20736d4c3443|2.868459105769945 |
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|0x80dad80653c154af:0x82b27cd1891ac768|2.9238454862870333|
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|0x89db8ebf3050a1cf:0x2337c904265a0c5d|2.974867107002594 |
|0x80c365b667f33bb1:0x1c2d5c7b4ccf567f|0x880e66c823c5a50d:0x69d2c1654369dfb8|2.8610219727032287|
|0x80c365b667f33bb1:0x1c2d5c7b

In [27]:
graph = GraphFrame(vertices, edges)
print(f"Graph created with {graph.vertices.count()} vertices and {graph.edges.count()} edges")


Graph created with 18646 vertices and 173827335 edges


In [ ]:
pagerank_results = graph.pageRank(resetProbability=0.15, maxIter=10)
top_neighborhoods_pr = pagerank_results.vertices.orderBy(
    F.col("pagerank").desc()
).select("id", "pagerank", "avg_success_score", "num_businesses", "avg_population")

print("\nTop Neighborhoods by PageRank (Network Importance):")
top_neighborhoods_pr.show(15, truncate=False)



Top Neighborhoods by PageRank (Network Importance):


+-------------------------------------+------------------+------------------+--------------+--------------+
|id                                   |pagerank          |avg_success_score |num_businesses|avg_population|
+-------------------------------------+------------------+------------------+--------------+--------------+
|0x89ef5527f3988829:0x949afafe59abd303|1110.581221446491 |3.141975406814372 |4             |4220.0        |
|0x89ef5526c5d34c9f:0x460a48183ec8d181|585.6566348042383 |3.1085309086321278|4             |4220.0        |
|0x89ef54a9c3e900ed:0xf06caf44e2db83c9|402.8141919915961 |3.214667611366157 |5             |4220.0        |
|0x89ef54a82e8285c3:0x8d21761cdd04226f|308.653518163305  |3.125872767770369 |5             |4220.0        |
|0x89ef54a819515555:0xdc0562632a8de19c|250.92416883881992|3.1700429287205147|3             |4220.0        |
|0x89ef5357d57242bd:0xdd086876bff13c80|211.790686880177  |3.1571643108507335|4             |4220.0        |
|0x89ef5357d3b767a1:0x25fc6e

In [30]:
checkpoint_dir = "/tmp/graphframes-checkpoints"
spark.sparkContext.setCheckpointDir(checkpoint_dir)

try:
    communities = graph.connectedComponents()
    community_summary = communities.groupBy("component").agg(
        F.count("*").alias("neighborhood_count"),
        F.avg("avg_success_score").alias("avg_community_success"),
        F.collect_list("id").alias("neighborhoods")
    )

    print("\nBusiness Communities Identified:")
    community_summary.orderBy(F.col("avg_community_success").desc()).show(10, truncate=False)
except Exception as e:
    print(f"Note: Connected Components analysis skipped due to: {e}")

25/12/02 04:53:24 WARN org.apache.spark.SparkContext: Spark is not running in local mode, therefore the checkpoint directory must not be on the local filesystem. Directory '/tmp/graphframes-checkpoints' appears to be on the local filesystem.



Business Communities Identified:


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
in_degrees = graph.inDegrees
out_degrees = graph.outDegrees

degree_centrality = in_degrees.join(
    out_degrees, 
    in_degrees.id == out_degrees.id, 
    "outer"
).select(
    F.coalesce(in_degrees.id, out_degrees.id).alias("id"),
    F.coalesce(in_degrees.inDegree, F.lit(0)).alias("inDegree"),
    F.coalesce(out_degrees.outDegree, F.lit(0)).alias("outDegree")
).withColumn("totalDegree", F.col("inDegree") + F.col("outDegree"))

# Join with neighborhood stats
hub_neighborhoods = degree_centrality.join(
    vertices, "id"
).orderBy(F.col("totalDegree").desc())

print("\nHub Neighborhoods (High Connectivity):")
hub_neighborhoods.select(
    "id", "totalDegree", "avg_success_score", "num_businesses"
).show(15, truncate=False)


Hub Neighborhoods (High Connectivity):


+-------------------------------------+-----------+------------------+--------------+
|id                                   |totalDegree|avg_success_score |num_businesses|
+-------------------------------------+-----------+------------------+--------------+
|0x4065fb08f951ce9f:0xb6719b0c44de8cf2|18645      |3.3450746046385786|8             |
|0x4cca389d788fd199:0xd367a2ccf3205600|18645      |3.196254010231819 |8             |
|0x4cca931bebced5b3:0x4a2e166f332d4252|18645      |3.1029220578969174|3             |
|0x4ccae261e4c51dab:0x4e9ccca84e526c14|18645      |3.155422127270397 |5             |
|0x4ccb1ad84f25369d:0x784e7288847d20a7|18645      |2.936281277892505 |3             |
|0x4ccdaa22509ca92d:0x90d0720f7b4641a9|18645      |3.1347882736997086|3             |
|0x54d292b6af8e7ce9:0x1ab283118056c771|18645      |3.338200159708503 |5             |
|0x808035790e7a0cad:0x14badc2acc9de2da|18645      |3.209651289817806 |4             |
|0x808158db89b30059:0x10e4005f523b0a9f|18645      |3.2

In [32]:
top_neighborhoods = vertices.orderBy(
    F.col("avg_success_score").desc()
).select("id").limit(10)

# Get businesses in these top neighborhoods
top_neighborhood_businesses = business_with_prob_df_clean.join(
    top_neighborhoods,
    business_with_prob_df_clean.neighborhood == top_neighborhoods.id
)


In [33]:
feature_cols = [c for c in business_with_prob_df_clean.columns if c.endswith("_flag")]

print("\nFeature Prevalence in Top Neighborhoods:")
feature_analysis_list = []
for feature in feature_cols:
    prevalence = top_neighborhood_businesses.agg(
        F.avg(feature).alias("prevalence")
    ).collect()[0]["prevalence"]
    
    if prevalence and prevalence > 0.1:  # Only show features with >10% prevalence
        feature_name = feature.replace("_flag", "").replace("MISC_", "").replace("_", " ")
        feature_analysis_list.append((feature_name, prevalence))

feature_analysis_list.sort(key=lambda x: x[1], reverse=True)
for feature, prev in feature_analysis_list[:20]:
    print(f"  • {feature}: {prev:.1%}")


Feature Prevalence in Top Neighborhoods:


  • Atmosphere Casual: 100.0%
  • Popular for Solo dining: 100.0%
  • Service options Takeout: 100.0%
  • Service options Delivery: 96.8%
  • Accessibility Wheelchair accessible entrance: 93.5%
  • Atmosphere Cozy: 90.3%
  • Offerings Comfort food: 87.1%
  • Dining options Dessert: 83.9%
  • Amenities Good for kids: 80.6%
  • Crowd Groups: 80.6%
  • Offerings Quick bite: 77.4%
  • Popular for Lunch: 77.4%
  • Service options Dine-in: 77.4%
  • Popular for Dinner: 67.7%
  • Offerings Coffee: 64.5%
  • Crowd Tourists: 58.1%
  • Amenities High chairs: 54.8%
  • Highlights Great coffee: 45.2%
  • Dining options Dinner: 32.3%
  • Planning Accepts reservations: 32.3%


In [34]:
recommendations = (
    pagerank_results.vertices
    .join(hub_neighborhoods.select("id", "totalDegree"), "id")
    .withColumn(
        "recommendation_score",
        (
            F.col("pagerank") * 0.3 +
            F.col("avg_success_score") * 0.5 +
            (F.col("totalDegree") / F.lit(100.0)) * 0.2  # Normalize degree
        )
    )
    .orderBy(F.col("recommendation_score").desc())
)

In [35]:
print("TOP RECOMMENDED NEIGHBORHOODS FOR NEW BUSINESS:")
recommendations.select(
    "id",
    "recommendation_score",
    "avg_success_score",
    "pagerank",
    "totalDegree",
    "num_businesses",
    "avg_population"
).show(10, truncate=False)


TOP RECOMMENDED NEIGHBORHOODS FOR NEW BUSINESS:


+-------------------------------------+--------------------+------------------+------------------+-----------+--------------+--------------+
|id                                   |recommendation_score|avg_success_score |pagerank          |totalDegree|num_businesses|avg_population|
+-------------------------------------+--------------------+------------------+------------------+-----------+--------------+--------------+
|0x89ef5527f3988829:0x949afafe59abd303|372.03535413735455  |3.1419754068143724|1110.581221446491 |18645      |4             |4220.0        |
|0x89ef5526c5d34c9f:0x460a48183ec8d181|214.54125589558754  |3.1085309086321278|585.6566348042383 |18645      |4             |4220.0        |
|0x89ef54a9c3e900ed:0xf06caf44e2db83c9|159.7415914031619   |3.214667611366157 |402.8141919915961 |18645      |5             |4220.0        |
|0x89ef54a82e8285c3:0x8d21761cdd04226f|131.4489918328767   |3.1258727677703684|308.653518163305  |18645      |5             |4220.0        |
|0x89ef54a819

In [38]:
top_recommendation = recommendations.select("id").first()["id"]

print(f"SUCCESS FEATURES FOR TOP NEIGHBORHOOD: {top_recommendation}")
top_rec_businesses = business_with_prob_df_clean.filter(
    F.col("neighborhood") == top_recommendation
)

# Most common features
print("\nMost Common Service/Operational Features:")
feature_analysis = []
for feature in feature_cols:
    prevalence = top_rec_businesses.agg(
        F.avg(feature).alias("prevalence")
    ).collect()[0]["prevalence"]
    
    if prevalence and prevalence > 0.15:
        feature_name = feature.replace("_flag", "").replace("MISC_", "").replace("_", " ")
        feature_analysis.append((feature_name, prevalence))

feature_analysis.sort(key=lambda x: x[1], reverse=True)
for feature, prev in feature_analysis[:15]:
    print(f"  • {feature}: {prev:.1%}")

SUCCESS FEATURES FOR TOP NEIGHBORHOOD: 0x89ef5527f3988829:0x949afafe59abd303

Most Common Service/Operational Features:
  • Amenities Good for kids: 75.0%
  • Amenities Restroom: 75.0%
  • Atmosphere Casual: 75.0%
  • Atmosphere Cozy: 75.0%
  • Crowd Groups: 75.0%
  • Crowd Tourists: 75.0%
  • Offerings Comfort food: 75.0%
  • Offerings Quick bite: 75.0%
  • Popular for Solo dining: 75.0%
  • Popular for Lunch: 75.0%
  • Popular for Dinner: 75.0%
  • Service options Delivery: 75.0%
  • Service options Takeout: 75.0%
  • Accessibility Wheelchair accessible entrance: 50.0%
  • Amenities High chairs: 50.0%


In [39]:
top_5_neighborhoods = recommendations.select("id").limit(5)

top_5_businesses = business_with_prob_df_clean.join(
    top_5_neighborhoods,
    business_with_prob_df_clean.neighborhood == top_5_neighborhoods.id
)

In [41]:
category_analysis = (
    top_5_businesses
    .select(F.explode(F.col("category")).alias("category"))
    .groupBy("category")
    .count()
    .orderBy(F.col("count").desc())
)

print("\nMost Common Restaurant Categories in Top Neighborhoods:")
category_analysis.show(20, truncate=False)


Most Common Restaurant Categories in Top Neighborhoods:


+-------------------------------+-----+
|category                       |count|
+-------------------------------+-----+
|Restaurant                     |14   |
|Seafood restaurant             |10   |
|Bar                            |9    |
|American restaurant            |6    |
|New American restaurant        |2    |
|Sushi restaurant               |2    |
|Traditional American restaurant|1    |
|Breakfast restaurant           |1    |
|Night club                     |1    |
|Gastropub                      |1    |
|Sandwich shop                  |1    |
|Irish pub                      |1    |
|Farmers' market                |1    |
|Event venue                    |1    |
|Bar & grill                    |1    |
|European restaurant            |1    |
|Caterer                        |1    |
|Mexican restaurant             |1    |
|Mediterranean restaurant       |1    |
|German restaurant              |1    |
+-------------------------------+-----+
only showing top 20 rows

